# Exploratory: method to extract data from kml file in Python

In this script we seek to extract refinery data from a google earth keyhole markup language (kml) file and store that in a Python dataframe. Once the data is captured we aim to remove duplicate records and append country, state and city data to each gps coordinate. 

Our thought is to try and see one part of the oil and gas industry (refineries - downstream) from a country perspective and for larger countries from a state by state perspective. And in so doing attempt to reduce complexity and better understand linkages between states that produce oil and countries that import oil. Further down the track aim to identify those refineries that carry out further processing of refined crude and gas and deliver petrochemical feedstocks such as ethyelene and propylene.

The data source is a google earth [link](https://www.google.com/maps/d/u/0/edit?mid=1OyFZs60ZesOe7RZaBwoU6eG4qFMO1Cg&usp=sharing) to a selection of world refineries that has been downloaded and saved in a file called core3.kml. 


## Set up the packages

In [1]:
from pykml import parser
from lxml import etree
import pandas as pd
from geopy.geocoders import Nominatim
import time
import numpy as np

## Prepare to read in kml file

In [2]:
# Define the namespace map to handle the KML namespace
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# Open the KML file
with open('core3.kml', 'rt', encoding="utf-8") as myfile:
    doc = parser.parse(myfile).getroot()


## Extract data 

In [3]:
# Extract the data from the KML file
data = []
for placemark in doc.xpath('.//kml:Placemark', namespaces=ns):
    name = placemark.find('kml:name', ns).text if placemark.find('kml:name', ns) is not None else None
    description = placemark.find('kml:description', ns).text if placemark.find('kml:description', ns) is not None else None
    coordinates_element = placemark.find('.//kml:Point/kml:coordinates', ns)
    if coordinates_element is not None:
        # Extract coordinates and split by comma
        coordinates = coordinates_element.text.strip().split(',')
        if len(coordinates) >= 2:
            # Take only the first two coordinates (longitude, latitude), reverse them, and ensure they are floats
            longitude, latitude = map(float, coordinates[:2])
            reversed_coordinates = [latitude, longitude]
        else:
            reversed_coordinates = None
    else:
        reversed_coordinates = None
    
    data.append({'name': name, 'description': description, 'reversed_coordinates': reversed_coordinates})


## Store data in a data frame

In [4]:
# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

## Tidy data

In [5]:
# Filter out rows where 'reversed_coordinates' is None
filtered_df = df.dropna(subset=['reversed_coordinates'])

# Create separate columns for latitude and longitude from the filtered DataFrame
if not filtered_df.empty:
    coordinates_df = pd.DataFrame(filtered_df['reversed_coordinates'].tolist(), columns=['latitude', 'longitude'], index=filtered_df.index)
    filtered_df = filtered_df.assign(latitude=coordinates_df['latitude'], longitude=coordinates_df['longitude'])

print(filtered_df)

                                                   name  \
0                             Sonatrach Skikda Refinery   
1                              Sonatrach Arzew Refinery   
2                     Sonatrach El Harrach Refinery????   
3                     Sonatrach Hassi Messaoud Refinery   
4                                    NOC Zawia Refinery   
...                                                 ...   
1817     23 - ExxonMobil Refining + Supply Co. - Altona   
1818  24 - Shell Refining (Australia) Pty. Ltd. - Clyde   
1819  25 - Shell Refining (Australia) Pty. Ltd. - Ge...   
1820  343 - New Zealand Refining Co. Ltd. - Marsden ...   
1821                      359 - Interoil - Port Moresby   

                                            description  \
0                                           323,000 bpd   
1                                            54,000 bpd   
2                                            59,000 bpd   
3                                            27,000 bpd

## Describe data frame

In [6]:
# Row and Column count
print(f"Row count: {filtered_df.shape[0]}")
print(f"Column count: {filtered_df.shape[1]}")

# Column Data Types
print("\nColumn Data Types:")
print(filtered_df.dtypes)

Row count: 1821
Column count: 5

Column Data Types:
name                     object
description              object
reversed_coordinates     object
latitude                float64
longitude               float64
dtype: object


## Save working data

In [7]:
# Save the filtered DataFrame to a excel file
filtered_df.to_excel('output.xlsx', index=False)

## Append country, state and city details

In [8]:
# Initialize the geolocator
geolocator = Nominatim(user_agent="learning_app_getcountry")

def get_location_by_coordinates(lat, lon):
    location = geolocator.reverse([lat, lon], exactly_one=True, language="en")
    address = location.raw['address']
    city = address.get('city', '')
    state = address.get('state', '')
    country = address.get('country', '')
    print(f"Geocoded coordinates: {lat}, {lon} to {city}, {state}, {country}")
    time.sleep(1)   # Sleep for 1 second to avoid hitting the rate limit       
    return city, state, country

## Splitting geocoding into batches for smooth processing

```python
# Convert this markdown block into code to process

# Define batch size
batch_size = 100

# Create a list to store DataFrames
dfs = []

# Divide the DataFrame into batches
for i in np.arange(0, len(filtered_df), batch_size):
    batch_df = filtered_df.iloc[i:i+batch_size].copy()
    batch_df['city'], batch_df['state'], batch_df['country'] = zip(*batch_df.apply(lambda row: get_location_by_coordinates(row['latitude'], row['longitude']), axis=1))
    dfs.append(batch_df)
    # Save each batch to a separate Excel file
    batch_df.to_excel(f'output_batch_{i//batch_size + 1}.xlsx', index=False)

# Merge all batches into a single DataFrame
merged_df = pd.concat(dfs)


## Save the output to different file formats 

```python
# convert this markdown block into code to process

# Save the merged DataFrame to an Excel file
merged_df.to_excel('output_merged.xlsx', index=False)

# Save the merged DataFrame to a pickle file
merged_df.to_pickle('output_merged.pkl')

# Save the merged DataFrame to a CSV file
merged_df.to_csv('output_merged.csv', index=False)

# Save the merged DataFrame to a JSON file
merged_df.to_json('output_merged.json', orient='records')

# Save the merged DataFrame to a HTML file
merged_df.to_html('output_merged.html', index=False)

